In [2]:
import cv2
from time import time
import numpy as np
from ultralytics.solutions.solutions import BaseSolution
from ultralytics.utils.plotting import Annotator, colors
from datetime import datetime
from paddleocr import PaddleOCR
import pandas as pd
from ultralytics import YOLO

In [3]:
model = YOLO(r"D:\Prabha\Cubeai_intern\Construction\Source_codes\Number Plate Recognition (NPR)\runs\detect\train\weights\best.pt")

In [4]:
video = r"D:\Prabha\Cubeai_intern\Construction\Licence Plate Camera Illustration Video(1080P_HD).mp4"

In [5]:
class SpeedEstimator(BaseSolution):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.initialize_region()
        self.spd = {}
        self.trkd_ids = []
        self.trk_pt = {}
        self.trk_pp = {}
        self.logged_ids = set()
        self.ocr = PaddleOCR(use_angle_cls=True, lang='en')
        self.excel_file = "speed_estimation.xlsx"
        self.init_excel()

    def init_excel(self):
        columns = ["Date", "Time", "Track ID", "Class Name", "Speed", "Number Plate"]
        try:
            pd.read_excel(self.excel_file)
        except FileNotFoundError:
            pd.DataFrame(columns=columns).to_excel(self.excel_file, index=False)

    def perform_ocr(self, image_array):
        if image_array is None:
            raise ValueError("Image is None")
        if isinstance(image_array, np.ndarray):
            results = self.ocr.ocr(image_array, rec=True)
        else:
            raise TypeError("Input image is not a valid numpy array")
        return ' '.join([result[1][0] for result in results[0]] if results[0] else "")

    def save_to_excel(self, date, time, track_id, class_name, speed, numberplate):
        data = pd.DataFrame([[date, time, track_id, class_name, speed, numberplate]],
                            columns=["Date", "Time", "Track ID", "Class Name", "Speed", "Number Plate"])
        existing_data = pd.read_excel(self.excel_file)
        updated_data = pd.concat([existing_data, data], ignore_index=True)
        updated_data.to_excel(self.excel_file, index=False)

    def estimate_speed(self, im0):
        self.annotator = Annotator(im0, line_width=self.line_width)
        self.extract_tracks(im0)
        current_time = datetime.now()

        for box, track_id, cls in zip(self.boxes, self.track_ids, self.clss):
            self.store_tracking_history(track_id, box)

            if track_id not in self.trk_pt:
                self.trk_pt[track_id] = 0
            if track_id not in self.trk_pp:
                self.trk_pp[track_id] = self.track_line[-1]

            speed_label = f"{int(self.spd[track_id])} km/h" if track_id in self.spd else self.names[int(cls)]
            label = f"ID: {track_id} {speed_label}"
            self.annotator.box_label(box, label=label, color=colors(track_id, True))

            if self.LineString([self.trk_pp[track_id], self.track_line[-1]]).intersects(self.r_s):
                direction = "known"
            else:
                direction = "unknown"

            if direction == "known" and track_id not in self.trkd_ids:
                self.trkd_ids.append(track_id)
                time_difference = time() - self.trk_pt[track_id]
                if time_difference > 0:
                    speed = np.abs(self.track_line[-1][1].item() - self.trk_pp[track_id][1].item()) / time_difference
                    self.spd[track_id] = round(speed)

            self.trk_pt[track_id] = time()
            self.trk_pp[track_id] = self.track_line[-1]
            x1, y1, x2, y2 = map(int, box)
            cropped_image = np.array(im0)[y1:y2, x1:x2]
            ocr_text = self.perform_ocr(cropped_image)
            class_name = self.names[int(cls)]
            speed = self.spd.get(track_id)

            if track_id not in self.logged_ids and ocr_text.strip() and speed is not None:
                self.save_to_excel(
                    current_time.strftime("%Y-%m-%d"),
                    current_time.strftime("%H:%M:%S"),
                    track_id,
                    class_name,
                    speed if speed is not None else 0.0,
                    ocr_text
                )
                self.logged_ids.add(track_id)

        self.display_output(im0)
        return im0
    
    def RGB(self, event, x, y, flags, param):
        if event == cv2.EVENT_MOUSEMOVE:
            point = [x, y]

cap = cv2.VideoCapture(video)
region_points = [(0, 315), (1018, 315)]
speed_obj = SpeedEstimator(
    region=region_points,
    model=r"D:\Prabha\Cubeai_intern\Construction\Source_codes\Number Plate Recognition (NPR)\runs\detect\train\weights\best.pt",
    line_width=2
)

count = 0
cv2.namedWindow("RGB")
cv2.setMouseCallback("RGB", speed_obj.RGB)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    count += 1
    if count % 3 != 0:
        continue

    frame = cv2.resize(frame, (1020, 500))
    result = speed_obj.estimate_speed(frame)
    cv2.imshow("RGB", result)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Ultralytics Solutions:  {'region': [(0, 315), (1018, 315)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'model': 'D:\\Prabha\\Cubeai_intern\\Construction\\Source_codes\\Number Plate Recognition (NPR)\\runs\\detect\\train\\weights\\best.pt', 'line_width': 2}
[2025/03/24 15:33:33] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Jarvis/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=